# パラメタ定義

In [111]:
# パラメタ名, グループID, データ型(C++), データ型(Optuna), レンジ下限/上限(or 100%分割)
param_def_list = [
    ('MoveDelInsert', 'Grp01', 'int', 'split', -1, -1),
    ('MoveRangeSwap', 'Grp01', 'int', 'split', -1, -1),
    ('MoveRangeEndSwap', 'Grp01', 'int', 'split', -1, -1),
    ('MoveMoveAdjSwap', 'Grp01', 'int', 'split', -1, -1),
    ('RangeSwapMaxWidth', 'Grp02', 'int', 'int', 5, 100),
]

In [112]:
def get_snake_case(param_name):
    snake_str = ''
    
    for i in range(len(param_name)):
        c = param_name[i]
        if c.isupper():
            if i > 0:
                snake_str += '_'
                
        snake_str += c.lower()
        
    return snake_str

# Parameter.cpp/Parameter.hpp用

In [113]:
# 変数定義
for param_namme, _, type_name, _, _, _ in param_def_list:
    print('{} {};'.format(type_name, get_snake_case(param_namme) + '_'))


int move_del_insert_;
int move_range_end_swap_;
int range_swap_max_width_;


In [114]:
# 関数宣言
for param_namme, _, type_name, _, _, _ in param_def_list:
    print('{} Get{}() const;'.format(type_name, param_namme))
    print('void Set{}({} val);'.format(param_namme, type_name))
    print()


int GetMoveDelInsert() const;
void SetMoveDelInsert(int val);

int GetMoveRangeEndSwap() const;
void SetMoveRangeEndSwap(int val);

int GetRangeSwapMaxWidth() const;
void SetRangeSwapMaxWidth(int val);



In [115]:
group_elems_dict = {}

for param_namme, group_name, _, _, _, _ in param_def_list:
    if group_name not in group_elems_dict.keys():
        group_elems_dict[group_name] = []
        
    group_elems_dict[group_name].append(param_namme)

In [116]:
# 変数初期値の定義
for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    if py_type_name == 'split':
        val = 100 // len(group_elems_dict[group_name])
    else:
        val = (val_lb + val_ub) / 2

    print('{}({}),'.format(get_snake_case(param_namme) + '_', val))

move_del_insert_(50),
move_range_end_swap_(50),
range_swap_max_width_(52.5),


In [117]:
# 関数定義
for param_namme, _, type_name, _, _, _ in param_def_list:
    print('{} Parameter::Get{}() const {{'.format(type_name, param_namme))
    print('   return {};'.format(get_snake_case(param_namme) + '_'))
    print('}')
    print()
    
    print('void Parameter::Set{}({} val) {{'.format(param_namme, type_name))
    print('   {} = val;'.format(get_snake_case(param_namme) + '_'))
    print('}')
    print()


int Parameter::GetMoveDelInsert() const {
   return move_del_insert_;
}

void Parameter::SetMoveDelInsert(int val) {
   move_del_insert_ = val;
}

int Parameter::GetMoveRangeEndSwap() const {
   return move_range_end_swap_;
}

void Parameter::SetMoveRangeEndSwap(int val) {
   move_range_end_swap_ = val;
}

int Parameter::GetRangeSwapMaxWidth() const {
   return range_swap_max_width_;
}

void Parameter::SetRangeSwapMaxWidth(int val) {
   range_swap_max_width_ = val;
}



# Optuna用バイナリ

In [118]:
print('   // -- param input start --')

for param_namme, _, type_name, _, _, _ in param_def_list:
    print('   {} {};'.format(type_name, get_snake_case(param_namme)))
    print('   cin >> {};'.format(get_snake_case(param_namme)))
    print('   param.Set{}({});'.format(param_namme, get_snake_case(param_namme)))

    print()
print('   // -- param input end --')

   // -- param input start --
   int move_del_insert;
   cin >> move_del_insert;
   param.SetMoveDelInsert(move_del_insert);

   int move_range_end_swap;
   cin >> move_range_end_swap;
   param.SetMoveRangeEndSwap(move_range_end_swap);

   int range_swap_max_width;
   cin >> range_swap_max_width;
   param.SetRangeSwapMaxWidth(range_swap_max_width);

   // -- param input end --


# Optuna用Notebook

In [119]:
print('init_param_list = [')
for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    if py_type_name == 'split':
        val = 100 // len(group_elems_dict[group_name])
    else:
        val = (val_lb + val_ub) // 2

    print('\t{},'.format(val))
print(']')

init_param_list = [
	50,
	50,
	52,
]


In [121]:
add_prefix = lambda x: 'x_' + x

# trial定義
for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    t = py_type_name
    snake_name = get_snake_case(param_namme)
    
    if t == 'split':
        snake_name = 'u_' + snake_name
        t = 'float'
        val_lb = 0.0001
        val_ub = 1.0
        
    print("    {} = trial.suggest_{}('{}', {}, {})".format(snake_name, t, snake_name, val_lb, val_ub))

print()

# ディリクレ分布への変換
for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    snake_name = get_snake_case(param_namme)
    
    if py_type_name == 'split':
        print('    {} = -np.log({})'.format('x_' + snake_name, 'u_' + snake_name))

for group_name, elem_list in group_elems_dict.items():
    if len(elem_list) == 1:
        continue

    sum_str = ' + '.join(map(add_prefix, map(get_snake_case, elem_list)))
    print('    {} = {}'.format(get_snake_case(group_name), sum_str))
print()

for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    snake_name = get_snake_case(param_namme)
    
    if py_type_name == 'split':
        print('    {} = round(100 * {} / {})'.format(snake_name, 'x_' + snake_name, get_snake_case(group_name)))

print()
print('    param_list = []')

for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    snake_name = get_snake_case(param_namme)
    print('    param_list.append({})'.format(snake_name))

    u_move_del_insert = trial.suggest_float('u_move_del_insert', 0.0001, 1.0)
    u_move_range_end_swap = trial.suggest_float('u_move_range_end_swap', 0.0001, 1.0)
    range_swap_max_width = trial.suggest_int('range_swap_max_width', 5, 100)

    x_move_del_insert = -np.log(u_move_del_insert)
    x_move_range_end_swap = -np.log(u_move_range_end_swap)
    grp01 = x_move_del_insert + x_move_range_end_swap

    move_del_insert = round(100 * x_move_del_insert / grp01)
    move_range_end_swap = round(100 * x_move_range_end_swap / grp01)

    param_list = []
    param_list.append(move_del_insert)
    param_list.append(move_range_end_swap)
    param_list.append(range_swap_max_width)


In [123]:
print('study.enqueue_trial({')

for i in range(len(param_def_list)):
    param_namme = param_def_list[i][0]
    snake_name = get_snake_case(param_namme)

    print("    '{}': init_param_list[{}],".format(snake_name, i))

print('})')

study.enqueue_trial({
    'move_del_insert': init_param_list[0],
    'move_range_end_swap': init_param_list[1],
    'range_swap_max_width': init_param_list[2],
})


In [124]:
# [0, 100)変換
for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    snake_name = get_snake_case(param_namme)
    
    if py_type_name == 'split':
        print('{} = -np.log(study.best_params["{}"])'.format('x_' + snake_name, 'u_' + snake_name))

for group_name, elem_list in group_elems_dict.items():
    if len(elem_list) == 1:
        continue

    sum_str = ' + '.join(map(add_prefix, map(get_snake_case, elem_list)))
    print('{} = {}'.format(get_snake_case(group_name), sum_str))
print()

for param_namme, group_name, type_name, py_type_name, val_lb, val_ub in param_def_list:
    snake_name = get_snake_case(param_namme)
    
    if py_type_name == 'split':
        print('print("{} = {{}}".format(round(100 * x_{} / {})))'.format(snake_name, snake_name, get_snake_case(group_name)))

x_move_del_insert = -np.log(study.best_params["u_move_del_insert"])
x_move_range_end_swap = -np.log(study.best_params["u_move_range_end_swap"])
grp01 = x_move_del_insert + x_move_range_end_swap

print("move_del_insert = {}".format(round(100 * x_move_del_insert / grp01)))
print("move_range_end_swap = {}".format(round(100 * x_move_range_end_swap / grp01)))
